# Installing dependencies:-

In [ ]:
%pip install -q google-generativeai chromadb python-dotenv

# Import and config:-

In [ ]:
import google.generativeai as genai
import chromadb
import chromadb.utils.embedding_functions as ef
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = 'YOUR_API_KEY_HERE'
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
documents = [
    "Python is a high-level programming language known for its simplicity.",
    "JavaScript is primarily used for web development and interactive UIs.",
    "Machine learning enables computers to learn from data without explicit programming.",
    "React is a JavaScript library for building user interfaces.",
    "Data science combines statistics, programming, and domain expertise."
]

In [ ]:
client = chromadb.Client()
embedding_function = ef.GoogleGenerativeAiEmbeddingFunction(
    api_key=GOOGLE_API_KEY,
    model_name="text-embedding-004"
)
collection = client.get_or_create_collection(
    name="documents",
    embedding_function=embedding_function
)

for i, doc in enumerate(documents):
    collection.add(
        documents=[doc],
        ids=[f"doc_{i}"]
    )

In [ ]:
def search_documents(query, n_results=2):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    return results['documents'][0]

In [ ]:
def generate_response(query, context):
    model = genai.GenerativeModel('gemini-flash-latest')
    
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {query}

Answer:"""
    
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def rag_query(query):
    relevant_docs = search_documents(query)
    context = "\n".join(relevant_docs)
    answer = generate_response(query, context)
    
    print(f"Question: {query}\n")
    print(f"Context:\n{context}\n")
    print(f"Answer:\n{answer}")
    
    return answer

In [ ]:
rag_query("What is Python?")

In [ ]:
rag_query("How does machine learning work?")

In [ ]:
rag_query("Tell me about React")